In [ ]:
from dataloader import *


In [ ]:
os.listdir()


In [ ]:
os.listdir("../")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
tmp = dataloader("../data/clover")
imu, rotor, mocap = tmp.runPipeline()


In [ ]:
imu


In [ ]:
mocap


In [ ]:
x, y, z = mocap['pose.position.x'].to_numpy(
), mocap['pose.position.y'].to_numpy(), mocap['pose.position.z'].to_numpy()
fig4 = plt.figure()
ax = fig4.add_subplot(projection='3d')

# ax.view_init(elev=30, azim=-45)
ax.set_xlabel('x position, m')
ax.set_ylabel('y position, m')
ax.set_zlabel('z position, m')
ax.plot3D(x, y, z, '.', markersize=.1)
plt.show()


m1 = rotor['rpm_0']
plt.figure()
plt.plot(m1)
plt.show()
plt.figure()
plt.plot(z)
plt.show()


In [ ]:
tmp.homogenizeData()
tmp.ConcatData.plot(subplots=True, figsize=(20, 20))


In [ ]:
tmp.ConcatData


In [ ]:
data = tmp.ConcatData.copy()


In [ ]:
data


In [ ]:
quat_array = np.vstack((data['pose.orientation.w'].to_numpy(), data['pose.orientation.x'].to_numpy(
), data['pose.orientation.y'].to_numpy(), data['pose.orientation.z'].to_numpy())).T


In [ ]:
import numpy_quaternion
quaternion.as_quat_array(quat_array)


In [ ]:
a = np.random.random((5, 4))
a


In [ ]:
np.linalg.norm(a[0])


In [ ]:
np.linalg.norm(a, axis=1)
